# **Entrega 3 - SenecaLibre**

## **Integrantes**:

- Juan Manuel Jáuregui Rozo - 201922481

En este notebook se presentará una solución del caso 3 utilizando Pyomo a partir de la propuesta formal entregada anteriormente. En este caso, se presentarán las siguientes secciones:

- **Cambios en el Modelo**: Aquí se presentarán las nuevas consideraciones que se aplicaron con el fin de cumplir con el modelado del problema.

- **Caso 3: Gestión de Oferta en Centros de Distribución**: Aquí se presentará el modelado del caso 3. Se incluirá un archivo CSV de rutas, el valor de la función objetivo, un análisis de costos, la visualización de las rutas y un análisis de resultados.

## **Cambios en el modelo matemático**

En este caso, se realizaron una serie de cambios en el modelo propuesto inicialmente. En primer lugar, es importante aclarar que el objetivo principal de la propuesta si se está cumpliendo. El objetivo se basa en la creación de nuevos centros de distribución y esta implementación también considera eso. Sin embargo, en la propuesta se proponían los siguientes conjuntos:

- V: Conjunto de Vehículos
- A: Conjunto de Centros de Distribución
- P: Conjunto de Puntos de Entrega
- E: Conjunto de Estaciones de Recarga

En la implementación, los primeros 3 fueron considerados, tomando puntos de entrega como clientes. Sin embargo, la parte de estaciones de recarga se simplificó ya que el caso de recarga de nodos fue asignado como bono. Por otro lado, la variable de decisión si se mantuvo como se propuso inicialmente al igual que el cálculo de los costos incluidos en la función objetivo. Esta última fue modificada levemente al quitarle la consideración del costo de nodos de recarga. En este caso, la función objetivo quedaría de la siguiente forma:

$$
\sum_{i \in \text{nodes}} \sum_{j \in \text{nodes}} \sum_{k \in \text{vehicles}} \left( \text{distance\_cost}_{i,j,k} \cdot x_{i,j,k} \right) + \left( \text{duration\_cost}_{i,j,k} \cdot x_{i,j,k} \right)
$$

En esta parte se optimizan los costos de distancia y tiempo ya que son los costos que dependen de los vehículos y los nodos. El costo de mantenimiento y de carga que se consideran en esta implementación son vectores que no dependen de estos otros parámetros por lo cuál se suman al final como un costo fijo.

En términos de las restricciones, se consideraron las mismas que se muestran en la propuesta y se añadieron otras como la de salir de un depósito inicialmente y terminar en un depósito ya que esto hace que sea coherente el recorrido. Además, se añadió otra restricción de uso de vehículos (más del 80%) para garantizar la completitud del recorrido de los nodos.

## **Caso 3: Gestión de Oferta en Centros de Distribución**

En este caso 3 se tienen datos de clientes, drones, vehículos EV y vehículos de combustión interna (gasolina). Con estos datos se busca modelar la solución a partir de lo propuesto inicialmente: implementar doce (12) nuevos centros de distribución que estén ubicados estratégicamente. Dentro de este caso se consideran los siguientes supuestos:

- Solo 9 clientes con demandas específicas.
- Restricción adicional de overflow en centros hasta un 50%.
- Relación cliente-vehículo ajustada a 1.5:1.

**Objetivo**: Evaluar la capacidad del modelo para manejar limitaciones de oferta sin comprometer la factibilidad de las soluciones.

## **Implementación del modelo**

In [54]:
import pandas as pd
import numpy as np
import requests
import xml.etree.ElementTree as ET
from tqdm import tqdm
import plotly.express as px
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

# Load data
depots_df = pd.read_csv('../case_3_supply_limits/Depots.csv')
clients_df = pd.read_csv('../case_3_supply_limits/Clients.csv')
vehicles_df = pd.read_csv('../case_3_supply_limits/Vehicles.csv')
depots_capacity_df = pd.read_csv('../case_3_supply_limits/DepotCapacities.csv')

# Get coordinates
depots_coords = depots_df[['Longitude', 'Latitude']].values.tolist()
clients_coords = clients_df[['Longitude', 'Latitude']].values.tolist()

# Create vehicle ID in DataFrame
vehicles_df['VehicleID'] = vehicles_df.index

# Get vehicle data
vehicle_ids = vehicles_df['VehicleID'].values.tolist()

vehicle_ids = [id for id in vehicle_ids]

vehicle_types = vehicles_df['VehicleType'].values.tolist()

vehicle_types = [vtype for vtype in vehicle_types]

vehicle_ranges = vehicles_df['Range'].values.tolist()

vehicle_ranges = [r * 1000 for r in vehicle_ranges]

vehicle_capacities = vehicles_df['Capacity'].values.tolist()

vehicle_capacities = [capacity for capacity in vehicle_capacities]

# Create the vehicles dictionary
vehicles = {id: (vtype, range, capacity) for id, vtype, range, capacity in zip(vehicle_ids, vehicle_types, vehicle_ranges, vehicle_capacities)}

# Get the client demands
client_demands = clients_df['Product'].values.tolist()
client_demands = [demand for demand in client_demands]

# For all depots add demand in clients_demands at the beginning
depots_demands = [0] * len(depots_df)
client_demands = depots_demands + client_demands

# Get depot capacities
depot_capacities = depots_capacity_df['Product'].values.tolist()
depot_capacities = [capacity for capacity in depot_capacities]
client_capacities = [0] * len(clients_df)
depot_capacities = depot_capacities + client_capacities

# Get the total number of products
num_products = sum(client_demands)

# Get the coordinates of all the depots and clients
all_coords = depots_coords + clients_coords

# Create a string with all the coordinates
coords_str = ';'.join([f"{lon},{lat}" for lon, lat in all_coords])

# Create IDs for all depots and clients
depots_id = depots_df['LocationID'].values.tolist()
clients_id = clients_df['LocationID'].values.tolist()

In [51]:
# API URL
url = f"https://router.project-osrm.org/table/v1/driving/{coords_str}"

# Parameters
params = {
    'sources': ';'.join(map(str, range(len(all_coords)))),
    'destinations': ';'.join(map(str, range(len(all_coords)))),
    'annotations': 'duration,distance'
}

# Send the request
response = requests.get(url, params=params)

# Check for successful response
if response.status_code != 200:
    print(f"Error: {response.status_code}")
    print(response.text)
    exit()

data = response.json()

# Extract the OSRM distance matrix
OSRM_matrix = np.array(data['distances'])

# Extract the duration matrix
duration_matrix = np.array(data['durations'])

# Create the HAVERSINE distance matrix
def haversine(lon1, lat1, lon2, lat2):
    R = 6371.0
    dlon = np.radians(lon2 - lon1)
    dlat = np.radians(lat2 - lat1)
    a = np.sin(dlat / 2) * np.sin(dlat / 2) + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) * np.sin(dlon / 2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

haversine_matrix = np.zeros((len(all_coords), len(all_coords)))

for i in range(len(all_coords)):
    for j in range(len(all_coords)):
        haversine_matrix[i, j] = haversine(all_coords[i][0], all_coords[i][1], all_coords[j][0], all_coords[j][1])

In [52]:
# Calculate costs

# Create the cost matrix
distance_cost_matrix = np.zeros((len(all_coords), len(all_coords), len(vehicles)))
duration_cost_matrix = np.zeros((len(all_coords), len(all_coords), len(vehicles)))

# Create distance matrix
distance_matrix_3D = np.zeros((len(all_coords), len(all_coords), len(vehicles)))

# Calculate the cost matrix
for k in range(len(vehicles)):
    for i in range(len(all_coords)):
        for j in range(len(all_coords)):

            # Calculate the distance and time cost
            if vehicles[k][0] == 'Gas Car': 
                distance_cost_matrix[i, j, k] = (OSRM_matrix[i, j] * 5000) / 1000
                duration_cost_matrix[i, j, k] += (duration_matrix[i, j] * 500) / 60
                distance_matrix_3D[i, j, k] = OSRM_matrix[i, j]
            elif vehicles[k][0] == 'EV':
                distance_cost_matrix[i, j, k] = (OSRM_matrix[i, j] * 4000) / 1000
                duration_cost_matrix[i, j, k] += (duration_matrix[i, j] * 500) / 60
                distance_matrix_3D[i, j, k] = OSRM_matrix[i, j]
            else:
                distance_cost_matrix[i, j, k] = (haversine_matrix[i, j] * 500) / 1000
                duration_cost_matrix[i, j, k] = (haversine_matrix[i, j] / 1000) / 11.11
                distance_matrix_3D[i, j, k] = haversine_matrix[i, j]

# Calculate the maintenance cost vector
maintenance_cost = np.zeros(len(vehicles))

for vehicle in vehicles:
    if vehicles[vehicle][0] == 'Gas Car':
        maintenance_cost[vehicle] = 30000
    elif vehicles[vehicle][0] == 'EV':
        maintenance_cost[vehicle] = 21000
    else:
        maintenance_cost[vehicle] = 3000

# Calculate the loading cost vector
loading_cost = 0
total_demand = sum(client_demands)
loading_cost = total_demand * 100

In [ ]:
# Create the model
from amplpy import modules

model = ConcreteModel()

# Sets
model.nodes = Set(initialize=pyo.RangeSet(1, len(all_coords)))
model.clients = Set(initialize=pyo.RangeSet(clients_id[0], clients_id[-1]))
model.depots = Set(initialize=pyo.RangeSet(depots_id[0], depots_id[-1]))
model.vehicles = Set(initialize=pyo.RangeSet(1, len(vehicle_ranges)))

# Parameters (data)
model.demand = Param(model.nodes, initialize=lambda model, i: client_demands[i-1])
model.capacity = Param(model.vehicles, initialize=lambda model, i: vehicle_capacities[i-1])
model.range = Param(model.vehicles, initialize=lambda model, i: vehicle_ranges[i-1])
model.depot_capacity = Param(model.nodes, initialize=lambda model, i: depot_capacities[i-1])

# Parameters (matrices)
model.distance_cost = Param(model.nodes, model.nodes, model.vehicles, initialize=lambda model, i, j, k: distance_cost_matrix[i-1, j-1, k-1])
model.duration_cost = Param(model.nodes, model.nodes, model.vehicles, initialize=lambda model, i, j, k: duration_cost_matrix[i-1, j-1, k-1])
model.maintenance_cost = Param(model.vehicles, initialize=lambda model, i: maintenance_cost[i-1])
model.loading_cost = Param(initialize=lambda model: loading_cost)
model.distance_matrix_3D = Param(model.nodes, model.nodes, model.vehicles, initialize=lambda model, i, j, k: distance_matrix_3D[i-1, j-1, k-1])

# Variables
model.x = Var(model.nodes, model.nodes, model.vehicles, domain=Binary)

# Objective function
def obj_func(model):
    return sum(model.distance_cost[i, j, k] * model.x[i, j, k] for i in model.nodes for j in model.nodes for k in model.vehicles) +  sum(model.duration_cost[i, j, k] * model.x[i, j, k] for i in model.nodes for j in model.nodes for k in model.vehicles)
model.obj = Objective(rule=obj_func, sense=minimize)

# Constraints

# Flow balance constraint
def flow_balance_constraint(model, i, k):
    return sum(model.x[i, j, k] for j in model.nodes if j != i) == sum(model.x[j, i, k] for j in model.nodes if j != i)
model.flow_balance_constraint = Constraint(model.clients, model.vehicles, rule=flow_balance_constraint)

# Capacity constraint
def capacity_constraint(model, k):
    return sum(model.demand[i] * sum(model.x[i, j, k] for j in model.nodes if j != i) for i in model.clients) <= model.capacity[k]
model.capacity_constraint = Constraint(model.vehicles, rule=capacity_constraint)

# Range constraint
def range_constraint(model, k):
    return sum(model.distance_matrix_3D[i, j, k] * model.x[i, j, k] for i in model.nodes for j in model.nodes) <= model.range[k]
model.range_constraint = Constraint(model.vehicles, rule=range_constraint)

# Visit once constraint
def visit_once_constraint(model, i):
    return sum(model.x[i, j, k] for j in model.nodes for k in model.vehicles if j != i) == 1
model.visit_once_constraint = Constraint(model.clients, rule=visit_once_constraint)

# No self loop constraint
def no_self_loop_constraint(model, i, k):
    return model.x[i, i, k] == 0
model.no_self_loop_constraint = Constraint(model.nodes, model.vehicles, rule=no_self_loop_constraint)

# Subtour elimination constraint
model.u = Var(model.nodes, model.vehicles, domain=NonNegativeReals)

def subtour_elimination_constraint(model, i, j, k):
    if i != j and i not in model.depots and j not in model.depots:
        return model.u[i, k] - model.u[j, k] + len(model.nodes) * model.x[i, j, k] <= len(model.nodes) - 1
    return Constraint.Skip
model.subtour_elimination_constraint = Constraint(model.nodes, model.nodes, model.vehicles, rule=subtour_elimination_constraint)

# All vehicles must start from a depot constraint
def all_vehicles_from_depot_constraint(model, k):
    return sum(model.x[i, j, k] for i in model.depots for j in model.nodes if i != j) == 1
model.all_vehicles_from_depot_constraint = Constraint(model.vehicles, rule=all_vehicles_from_depot_constraint)

# All vehicles must end at a depot constraint
def all_vehicles_end_depot_constraint(model, k):
    return sum(model.x[i, j, k] for i in model.nodes for j in model.depots if i != j) == 1
model.all_vehicles_end_depot_constraint = Constraint(model.vehicles, rule=all_vehicles_end_depot_constraint)

# Vehicles must not go from depot to depot constraint
def no_depot_to_depot_constraint(model, i, j, k):
    if i in model.depots and j in model.depots:
        return model.x[i, j, k] == 0
    return Constraint.Skip
model.no_depot_to_depot_constraint = Constraint(model.nodes, model.nodes, model.vehicles, rule=no_depot_to_depot_constraint)

# Depot capacity constraint
def depot_capacity_constraint(model, j):
    total_demand_served_by_depot = sum(model.demand[i] * model.x[j, i, k] for i in model.clients for k in model.vehicles)
    return total_demand_served_by_depot <= model.depot_capacity[j]
model.depot_capacity_constraint = Constraint(model.depots, rule=depot_capacity_constraint)

In [79]:
def solve_model(model):
        solver_name = "appsi_highs"
        solver = pyo.SolverFactory(solver_name)
        solver.options['parallel'] = 'on'
        solver.options['time_limit'] = 3600/2  # 30 minutes time limit
        solver.options['presolve'] = 'on'
        solver.options['mip_rel_gap'] = 0.01  # 1% relative gap
        solver.options['simplex_strategy'] = 1  # Dual simplex
        solver.options['simplex_max_concurrency'] = 8  # Max concurrency
        solver.options['mip_min_logging_interval'] = 10  # Log every 10 seconds
        solver.options['mip_heuristic_effort'] = 0.2  # Increase heuristic effort

        result = solver.solve(model, tee=True)

        # Check solver status
        if result.solver.termination_condition == pyo.TerminationCondition.optimal:
            print("Optimal solution found.")
        elif result.solver.termination_condition == pyo.TerminationCondition.maxTimeLimit:
            print("Time limit reached, solution may be suboptimal.")
        else:
            print(f"Solver terminated with condition: {result.solver.termination_condition}")

        print(result)

solve_model(model)

Running HiGHS 1.8.1 (git hash: 4a7f24a): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [7e-01, 4e+04]
  Cost   [4e-01, 2e+05]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+06]
Presolving model
526 rows, 966 cols, 4704 nonzeros  0s
301 rows, 741 cols, 2949 nonzeros  0s
269 rows, 345 cols, 1358 nonzeros  0s

Solving MIP model with:
   269 rows
   345 cols (291 binary, 0 integer, 0 implied int., 54 continuous)
   1358 nonzeros
MIP-Timing:       0.022 - starting analytic centre calculation

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              G

RuntimeError: A feasible solution was not found, so no solution can be loaded. If using the appsi.solvers.Highs interface, you can set opt.config.load_solution=False. If using the environ.SolverFactory interface, you can set opt.solve(model, load_solutions = False). Then you can check results.termination_condition and results.best_feasible_objective before loading a solution.

In [74]:
for j in model.depots:
    for i in model.clients:
        for k in model.vehicles:
            if model.x[j, i, k].value == 1:
                print(f"Depot {j} serves Client {i} with Vehicle {k}")

Depot 5 serves Client 13 with Vehicle 4
Depot 5 serves Client 14 with Vehicle 2
Depot 9 serves Client 15 with Vehicle 1
Depot 9 serves Client 17 with Vehicle 5
Depot 9 serves Client 20 with Vehicle 6
Depot 11 serves Client 21 with Vehicle 3


## **Archivo CSV de Rutas**

In [75]:
# Create a CSV file with the routes. The file must have the following columns:
# - ID-Vehiculo: Vehicle ID
# - ID-Origen: Origin node ID
# - ID-Destino: Destination node ID
# Each row represents a route.

# Create DataFrame
routes_df = pd.DataFrame(columns=['ID-Vehiculo', 'ID-Origen', 'ID-Destino'])

# Get the routes
routes_list = []
for i in model.nodes:
    for j in model.nodes:
        for k in model.vehicles:
            if np.round(model.x[i, j, k].value) == 1:
                routes_list.append({'ID-Vehiculo': k, 'ID-Origen': i, 'ID-Destino': j})
                routes_list[-1]['Lon-Origen'] = all_coords[i-1][0]
                routes_list[-1]['Lat-Origen'] = all_coords[i-1][1]
                routes_list[-1]['Lon-Destino'] = all_coords[j-1][0]
                routes_list[-1]['Lat-Destino'] = all_coords[j-1][1]
                
                # Add type to the nodes if the first 12 nodes are depots and the rest are clients
                if 1 <= i <= 12:
                    routes_list[-1]['Type-Origen'] = 'Depot'
                else:
                    routes_list[-1]['Type-Origen'] = 'Client'
                
                if 1 <= j <= 12:
                    routes_list[-1]['Type-Destino'] = 'Depot'
                else:
                    routes_list[-1]['Type-Destino'] = 'Client'

# Add the routes to the DataFrame
routes_df = pd.concat([routes_df, pd.DataFrame(routes_list)], ignore_index=True)

# Order the DataFrame
routes_df = routes_df.sort_values(by=['ID-Vehiculo', 'ID-Origen'])

# Save the DataFrame to a CSV file
routes_df.to_csv('MOS3-caso-limites-3-ruta-extended.csv', index=False)

# Save the DataFrame to a CSV file with only the first 3 columns
routes_df[['ID-Vehiculo', 'ID-Origen', 'ID-Destino']].to_csv('MOS3-caso-limites-3-ruta.csv', index=False)

## **Valor de la función objetivo y análisis de costos**

In [43]:
# Create a report of the costs. The report must include the following information:
# - Total distance cost
# - Total duration cost
# - Total maintenance cost
# - Total loading cost

routes_df['Costo-Mantenimiento'] = routes_df.apply(lambda row: maintenance_cost[row['ID-Vehiculo']-1], axis=1)

# Calculate the total costs
total_loading_cost = float(model.loading_cost)
total_maintenance_cost = routes_df['Costo-Mantenimiento'].sum()

# Based on the value of the objective function, calculate the total distance and duration costs
total_distance_cost = 0
total_duration_cost = 0

for i in model.nodes:
    for j in model.nodes:
        for k in model.vehicles:
            total_distance_cost += model.distance_cost[i, j, k] * model.x[i, j, k].value
            total_duration_cost += model.duration_cost[i, j, k] * model.x[i, j, k].value

# Save the report to a text file
with open('MOS3-caso3_objetivo.txt', 'w') as f:
    f.write(f"Costo de distancia total: {total_distance_cost:.2f}\n")
    f.write(f"Costo de tiempo total: {total_duration_cost:.2f}\n")
    f.write(f"Costo de mantenimiento total: {total_maintenance_cost:.2f}\n")
    f.write(f"Costo de carga total: {total_loading_cost:.2f}\n")
    f.write(f"Valor de la función objetivo: {model.obj():.2f}\n")
    f.write(f"Costo total: {total_distance_cost + total_duration_cost + total_maintenance_cost + total_loading_cost:.2f}\n")

## **Visualización de rutas**

In [44]:
import folium
import pandas as pd
import numpy as np

# Load the routes DataFrame
routes_df = pd.read_csv('MOS3-caso-limites-3-ruta-extended.csv')

# Create a map centered at the average location of all origins and destinations
map_center = [
    routes_df[['Lat-Origen', 'Lat-Destino']].mean().mean(),
    routes_df[['Lon-Origen', 'Lon-Destino']].mean().mean()
]
mapa = folium.Map(location=map_center, zoom_start=12)

# Assign random colors for each vehicle
vehicle_ids = routes_df['ID-Vehiculo'].unique()
colors = {vehicle: f"#{''.join(np.random.choice(list('89ABCDEF'), size=6))}" for vehicle in vehicle_ids}

# Keep track of added origin markers to avoid duplicates
added_markers = set()

# Plot the routes
for _, row in routes_df.iterrows():
    # Draw route as a PolyLine
    folium.PolyLine(
        [(row['Lat-Origen'], row['Lon-Origen']), (row['Lat-Destino'], row['Lon-Destino'])],
        color=colors[row['ID-Vehiculo']],
        weight=5,
        opacity=0.8,
        tooltip=(
            f"Vehículo: {row['ID-Vehiculo']} | "
            f"Origen: {row['ID-Origen']} ({row['Type-Origen']}) → "
            f"Destino: {row['ID-Destino']} ({row['Type-Destino']})"
        )
    ).add_to(mapa)

    # Add a marker for the origin if it hasn't been added already
    origin_id = row['ID-Origen']
    if origin_id not in added_markers:
        marker_color = "blue" if row['Type-Origen'] == "Depot" else "green"
        folium.Marker(
            location=(row['Lat-Origen'], row['Lon-Origen']),
            popup=(
                f"Vehículo: {row['ID-Vehiculo']} | "
                f"Origen: {row['ID-Origen']} ({row['Type-Origen']})"
            ),
            icon=folium.Icon(color=marker_color, icon="info-sign")
        ).add_to(mapa)
        added_markers.add(origin_id)

# Save the map to an HTML file
mapa.save("MOS3-caso-limites-3-rutas.html")

## **Análisis de resultados**

A continuación, se presentará un análisis de los resultados obtenidos para este caso. Principalmente, se buscará observar cuál fue el vehículo más demorado, cuál fue el que recorrió una mayor distancia, cuál fue el que hizo el recorrido en menos tiempo y cuá fue el que tuvo un mejor rendimiento. De esta forma, se podrá entender un poco mejor cuál es el punto débil y el punto fuerte del modelado para así encontrar posibles mejoras a una futura implementación.

In [ ]:
# Obtain the vehicle that took longest to complete its route
results_df = pd.DataFrame(columns=['ID-Vehiculo', 'Duracion', 'Distancia'])

for vehicle in vehicle_ids:
    vehicle_routes = routes_df[routes_df['ID-Vehiculo'] == vehicle]
    vehicle_duration = 0
    vehicle_distance = 0

    for _, row in vehicle_routes.iterrows():
        vehicle_duration += duration_matrix[row['ID-Origen']-1, row['ID-Destino']-1]
        vehicle_distance += haversine_matrix[row['ID-Origen']-1, row['ID-Destino']-1]

    results_df = pd.concat([results_df, pd.DataFrame([{'ID-Vehiculo': vehicle, 'Duracion': vehicle_duration, 'Distancia': vehicle_distance}])], ignore_index=True)

# Get the vehicle that took the longest to complete its route
longest_vehicle = results_df.loc[results_df['Duracion'].idxmax()]

# Get the vehicle that took the shortest to complete its route
shortest_vehicle = results_df.loc[results_df['Duracion'].idxmin()]

# Get the vehicle that traveled the longest distance
longest_distance_vehicle = results_df.loc[results_df['Distancia'].idxmax()]

# Get the vehicle that traveled the shortest distance
shortest_distance_vehicle = results_df.loc[results_df['Distancia'].idxmin()]

# Save the results to a text file
with open('MOS3-caso-limites-3-resultados.txt', 'w') as f:
    f.write(f"Vehículo que tardó más en completar su ruta: {longest_vehicle['ID-Vehiculo']}\n")
    f.write(f"Duración: {longest_vehicle['Duracion']:.2f} minutos\n")
    f.write(f"Distancia: {longest_vehicle['Distancia']:.2f} km\n\n")

    f.write(f"Vehículo que tardó menos en completar su ruta: {shortest_vehicle['ID-Vehiculo']}\n")
    f.write(f"Duración: {shortest_vehicle['Duracion']:.2f} minutos\n")
    f.write(f"Distancia: {shortest_vehicle['Distancia']:.2f} km\n\n")

    f.write(f"Vehículo que recorrió la mayor distancia: {longest_distance_vehicle['ID-Vehiculo']}\n")
    f.write(f"Duración: {longest_distance_vehicle['Duracion']:.2f} minutos\n")
    f.write(f"Distancia: {longest_distance_vehicle['Distancia']:.2f} km\n\n")

    f.write(f"Vehículo que recorrió la menor distancia: {shortest_distance_vehicle['ID-Vehiculo']}\n")
    f.write(f"Duración: {shortest_distance_vehicle['Duracion']:.2f} minutos\n")
    f.write(f"Distancia: {shortest_distance_vehicle['Distancia']:.2f} km\n\n")